# AllOfUs

* **Project:** ADRD Genetic Diversity in Biobanks
* **Version:** Python/3.10
* **Last Updated:** 24-FEB-2024

## Notebook Overview
Create cohorts (AD, Dementia, Control), gene characterization, Genotools prep and application, allele freqs, APOE genotyping, ancestry definition, demographic data, resilience/protective variants

# Query All of Us to create three cohorts (AD, Dementia, and Control), and characterize the genes of interest

## Variables used 
- `${ANCESTRY}` = EUR, AFR, AMR, AAC, AJ, MDE, SAS, CAS, EAS, FIN, CAH
- `${COHORT}` = AD, Dementia, Control or Case, Control
- `${COUNT}` = Number of total individuals in each ancestry
- `${Gene}` = APP, PSEN1, PSEN2, GRN, MAPT, SNCA, GBA1, TREM2, TBK1, TARDBP, APOE

## AD Cohort

In [ ]:
import pandas
import os

#This query represents dataset "Alzheimer's disease and controls" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_34109098_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN  (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            select
                                cast(cr.id as string) as id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                            WHERE
                                concept_id IN (
                                    378419
                                ) 
                                AND full_text LIKE '%_rank1]%'
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    c_occurrence.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (
                                                SELECT
                                                    DISTINCT c.concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                JOIN
                                                    (
                                                        select
                                                            cast(cr.id as string) as id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                        WHERE
                                                            concept_id IN (378419) 
                                                            AND full_text LIKE '%_rank1]%'
                                                    ) a 
                                                        ON (
                                                            c.path LIKE CONCAT('%.',
                                                        a.id,
                                                        '.%') 
                                                        OR c.path LIKE CONCAT('%.',
                                                        a.id) 
                                                        OR c.path LIKE CONCAT(a.id,
                                                        '.%') 
                                                        OR c.path = a.id) 
                                                    WHERE
                                                        is_standard = 1 
                                                        AND is_selectable = 1
                                                    ) 
                                                    AND is_standard = 1 
                                            )
                                        ) criteria 
                                    ) 
                                    AND cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            has_whole_genome_variant = 1 
                                    )  
                                UNION
                                DISTINCT SELECT
                                    distinct person_id  
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                                WHERE
                                    cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            age_at_consent BETWEEN 65 AND 120 
                                    ) 
                                    AND cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            has_whole_genome_variant = 1 
                                    ) 
                                    AND cb_search_person.person_id NOT IN (
                                        SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    concept_id IN (
                                                        SELECT
                                                            DISTINCT c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    concept_id IN (372887) 
                                                                    AND full_text LIKE '%_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                is_standard = 1 
                                                                AND is_selectable = 1
                                                            ) 
                                                            AND is_standard = 1 
                                                    )
                                                ) criteria 
                                            ) 
                                            AND cb_search_person.person_id NOT IN (
                                                SELECT
                                                    criteria.person_id 
                                                FROM
                                                    (SELECT
                                                        DISTINCT person_id,
                                                        entry_date,
                                                        concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                    WHERE
                                                        (
                                                            concept_id IN (
                                                                SELECT
                                                                    DISTINCT c.concept_id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                JOIN
                                                                    (
                                                                        select
                                                                            cast(cr.id as string) as id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                        WHERE
                                                                            concept_id IN (376337) 
                                                                            AND full_text LIKE '%_rank1]%'
                                                                    ) a 
                                                                        ON (
                                                                            c.path LIKE CONCAT('%.',
                                                                        a.id,
                                                                        '.%') 
                                                                        OR c.path LIKE CONCAT('%.',
                                                                        a.id) 
                                                                        OR c.path LIKE CONCAT(a.id,
                                                                        '.%') 
                                                                        OR c.path = a.id) 
                                                                    WHERE
                                                                        is_standard = 1 
                                                                        AND is_selectable = 1
                                                                    ) 
                                                                    AND is_standard = 1 
                                                            )
                                                        ) criteria 
                                                    ) 
                                                    AND cb_search_person.person_id NOT IN (
                                                        SELECT
                                                            criteria.person_id 
                                                        FROM
                                                            (SELECT
                                                                DISTINCT person_id,
                                                                entry_date,
                                                                concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                            WHERE
                                                                (
                                                                    concept_id IN (
                                                                        SELECT
                                                                            DISTINCT c.concept_id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                        JOIN
                                                                            (
                                                                                select
                                                                                    cast(cr.id as string) as id 
                                                                                FROM
                                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                                WHERE
                                                                                    concept_id IN (4011630) 
                                                                                    AND full_text LIKE '%_rank1]%'
                                                                            ) a 
                                                                                ON (
                                                                                    c.path LIKE CONCAT('%.',
                                                                                a.id,
                                                                                '.%') 
                                                                                OR c.path LIKE CONCAT('%.',
                                                                                a.id) 
                                                                                OR c.path LIKE CONCAT(a.id,
                                                                                '.%') 
                                                                                OR c.path = a.id) 
                                                                            WHERE
                                                                                is_standard = 1 
                                                                                AND is_selectable = 1
                                                                            ) 
                                                                            AND is_standard = 1 
                                                                    )
                                                                ) criteria 
                                                            ) 
                                                            AND cb_search_person.person_id NOT IN (
                                                                SELECT
                                                                    criteria.person_id 
                                                                FROM
                                                                    (SELECT
                                                                        DISTINCT person_id,
                                                                        entry_date,
                                                                        concept_id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                                    WHERE
                                                                        (
                                                                            concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1740947) 
                                                                            OR  concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1740684) 
                                                                            OR  concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1384582) 
                                                                            OR  concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1740980)
                                                                        )) criteria ) 
                                                            )
                                                        )) c_occurrence 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                                                        ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                                                        ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                                        ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                                                        ON v.visit_concept_id = visit.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                                                        ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                                                        ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_34109098_condition_df = pandas.read_gbq(
    dataset_34109098_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_34109098_condition_df.head(5)

dataset_34109098_condition_df.to_csv("dataset_condition_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "Alzheimer's disease and controls" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_34109098_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                SELECT
                    DISTINCT concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (
                        select
                            cast(cr.id as string) as id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                        WHERE
                            concept_id IN (
                                1740639,1585855,1586134
                            ) 
                            AND domain_id = 'SURVEY'
                    ) a 
                        ON (
                            c.path like CONCAT('%',
                        a.id,
                        '.%')) 
                    WHERE
                        domain_id = 'SURVEY' 
                        AND type = 'PPI' 
                        AND subtype = 'QUESTION'
                    )
            )  
            AND (
                answer.PERSON_ID IN (
                    SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (378419) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 1 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 1 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_whole_genome_variant = 1 
                                )  
                            UNION
                            DISTINCT SELECT
                                distinct person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        age_at_consent BETWEEN 65 AND 120 
                                ) 
                                AND cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_whole_genome_variant = 1 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (372887) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (376337) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 1 
                                                        )
                                                    ) criteria 
                                                ) 
                                                AND cb_search_person.person_id NOT IN (
                                                    SELECT
                                                        criteria.person_id 
                                                    FROM
                                                        (SELECT
                                                            DISTINCT person_id,
                                                            entry_date,
                                                            concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                        WHERE
                                                            (
                                                                concept_id IN (
                                                                    SELECT
                                                                        DISTINCT c.concept_id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                    JOIN
                                                                        (
                                                                            select
                                                                                cast(cr.id as string) as id 
                                                                            FROM
                                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                            WHERE
                                                                                concept_id IN (4011630) 
                                                                                AND full_text LIKE '%_rank1]%'
                                                                        ) a 
                                                                            ON (
                                                                                c.path LIKE CONCAT('%.',
                                                                            a.id,
                                                                            '.%') 
                                                                            OR c.path LIKE CONCAT('%.',
                                                                            a.id) 
                                                                            OR c.path LIKE CONCAT(a.id,
                                                                            '.%') 
                                                                            OR c.path = a.id) 
                                                                        WHERE
                                                                            is_standard = 1 
                                                                            AND is_selectable = 1
                                                                        ) 
                                                                        AND is_standard = 1 
                                                                )
                                                            ) criteria 
                                                        ) 
                                                        AND cb_search_person.person_id NOT IN (
                                                            SELECT
                                                                criteria.person_id 
                                                            FROM
                                                                (SELECT
                                                                    DISTINCT person_id,
                                                                    entry_date,
                                                                    concept_id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                                WHERE
                                                                    (
                                                                        concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1740947) 
                                                                        OR  concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1740684) 
                                                                        OR  concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1384582) 
                                                                        OR  concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1740980)
                                                                    )) criteria ) 
                                                        )
                                                    )"""

dataset_34109098_survey_df = pandas.read_gbq(
    dataset_34109098_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_34109098_survey_df.head(5)
dataset_34109098_survey_df.to_csv("dataset_survey_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "Alzheimer's disease and controls" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_34109098_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (
                                    SELECT
                                        DISTINCT c.concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                    JOIN
                                        (
                                            select
                                                cast(cr.id as string) as id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                            WHERE
                                                concept_id IN (378419) 
                                                AND full_text LIKE '%_rank1]%'
                                        ) a 
                                            ON (
                                                c.path LIKE CONCAT('%.',
                                            a.id,
                                            '.%') 
                                            OR c.path LIKE CONCAT('%.',
                                            a.id) 
                                            OR c.path LIKE CONCAT(a.id,
                                            '.%') 
                                            OR c.path = a.id) 
                                        WHERE
                                            is_standard = 1 
                                            AND is_selectable = 1
                                        ) 
                                        AND is_standard = 1 
                                )
                            ) criteria 
                        ) 
                        AND cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_whole_genome_variant = 1 
                        )  
                    UNION
                    DISTINCT SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                age_at_consent BETWEEN 65 AND 120 
                        ) 
                        AND cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_whole_genome_variant = 1 
                        ) 
                        AND cb_search_person.person_id NOT IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (372887) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 1 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 1 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (376337) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (4011630) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 1 
                                                        )
                                                    ) criteria 
                                                ) 
                                                AND cb_search_person.person_id NOT IN (
                                                    SELECT
                                                        criteria.person_id 
                                                    FROM
                                                        (SELECT
                                                            DISTINCT person_id,
                                                            entry_date,
                                                            concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                        WHERE
                                                            (
                                                                concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1740947) 
                                                                OR  concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1740684) 
                                                                OR  concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1384582) 
                                                                OR  concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1740980)
                                                            )) criteria ) 
                                                )"""

dataset_34109098_person_df = pandas.read_gbq(
    dataset_34109098_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_34109098_person_df.head(5)
dataset_34109098_person_df.to_csv("dataset_person_df.csv", index=False)

In [ ]:
person_condition_df = pd.merge(left=dataset_34109098_person_df, right=dataset_34109098_condition_df)
person_condition_df.shape

person_condition_df.to_csv("person_condition_alz_df.csv", index=False)

!grep Alzheimer person_condition_alz_df.csv | cut -f1 -d "," | sort -u | uniq > person_condition_alz_df_uniqid

## Dementia Cohort

In [ ]:
import pandas
import os

# This query represents dataset "dementia and controls" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_58854988_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN  (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            select
                                cast(cr.id as string) as id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                            WHERE
                                concept_id IN (
                                    4182210
                                ) 
                                AND full_text LIKE '%_rank1]%'
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    c_occurrence.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (
                                                SELECT
                                                    DISTINCT c.concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                JOIN
                                                    (
                                                        select
                                                            cast(cr.id as string) as id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                        WHERE
                                                            concept_id IN (4182210) 
                                                            AND full_text LIKE '%_rank1]%'
                                                    ) a 
                                                        ON (
                                                            c.path LIKE CONCAT('%.',
                                                        a.id,
                                                        '.%') 
                                                        OR c.path LIKE CONCAT('%.',
                                                        a.id) 
                                                        OR c.path LIKE CONCAT(a.id,
                                                        '.%') 
                                                        OR c.path = a.id) 
                                                    WHERE
                                                        is_standard = 1 
                                                        AND is_selectable = 1
                                                    ) 
                                                    AND is_standard = 1 
                                            )
                                        ) criteria 
                                    ) 
                                    AND cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            has_whole_genome_variant = 1 
                                    )  
                                UNION
                                DISTINCT SELECT
                                    distinct person_id  
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                                WHERE
                                    cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            age_at_consent BETWEEN 65 AND 120 
                                    ) 
                                    AND cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            has_whole_genome_variant = 1 
                                    ) 
                                    AND cb_search_person.person_id NOT IN (
                                        SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    concept_id IN (
                                                        SELECT
                                                            DISTINCT c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    concept_id IN (372887) 
                                                                    AND full_text LIKE '%_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                is_standard = 1 
                                                                AND is_selectable = 1
                                                            ) 
                                                            AND is_standard = 1 
                                                    )
                                                ) criteria 
                                            ) 
                                            AND cb_search_person.person_id NOT IN (
                                                SELECT
                                                    criteria.person_id 
                                                FROM
                                                    (SELECT
                                                        DISTINCT person_id,
                                                        entry_date,
                                                        concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                    WHERE
                                                        (
                                                            concept_id IN (
                                                                SELECT
                                                                    DISTINCT c.concept_id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                JOIN
                                                                    (
                                                                        select
                                                                            cast(cr.id as string) as id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                        WHERE
                                                                            concept_id IN (376337) 
                                                                            AND full_text LIKE '%_rank1]%'
                                                                    ) a 
                                                                        ON (
                                                                            c.path LIKE CONCAT('%.',
                                                                        a.id,
                                                                        '.%') 
                                                                        OR c.path LIKE CONCAT('%.',
                                                                        a.id) 
                                                                        OR c.path LIKE CONCAT(a.id,
                                                                        '.%') 
                                                                        OR c.path = a.id) 
                                                                    WHERE
                                                                        is_standard = 1 
                                                                        AND is_selectable = 1
                                                                    ) 
                                                                    AND is_standard = 1 
                                                            )
                                                        ) criteria 
                                                    ) 
                                                    AND cb_search_person.person_id NOT IN (
                                                        SELECT
                                                            criteria.person_id 
                                                        FROM
                                                            (SELECT
                                                                DISTINCT person_id,
                                                                entry_date,
                                                                concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                            WHERE
                                                                (
                                                                    concept_id IN (
                                                                        SELECT
                                                                            DISTINCT c.concept_id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                        JOIN
                                                                            (
                                                                                select
                                                                                    cast(cr.id as string) as id 
                                                                                FROM
                                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                                WHERE
                                                                                    concept_id IN (4011630) 
                                                                                    AND full_text LIKE '%_rank1]%'
                                                                            ) a 
                                                                                ON (
                                                                                    c.path LIKE CONCAT('%.',
                                                                                a.id,
                                                                                '.%') 
                                                                                OR c.path LIKE CONCAT('%.',
                                                                                a.id) 
                                                                                OR c.path LIKE CONCAT(a.id,
                                                                                '.%') 
                                                                                OR c.path = a.id) 
                                                                            WHERE
                                                                                is_standard = 1 
                                                                                AND is_selectable = 1
                                                                            ) 
                                                                            AND is_standard = 1 
                                                                    )
                                                                ) criteria 
                                                            ) 
                                                            AND cb_search_person.person_id NOT IN (
                                                                SELECT
                                                                    criteria.person_id 
                                                                FROM
                                                                    (SELECT
                                                                        DISTINCT person_id,
                                                                        entry_date,
                                                                        concept_id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                                    WHERE
                                                                        (
                                                                            concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1740947) 
                                                                            OR  concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1740684) 
                                                                            OR  concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1384582) 
                                                                            OR  concept_id IN (1740558) 
                                                                            AND is_standard = 0  
                                                                            AND  value_source_concept_id IN (1740980)
                                                                        )) criteria ) 
                                                            )
                                                        )) c_occurrence 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                                                        ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                                                        ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                                        ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                                                        ON v.visit_concept_id = visit.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                                                        ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
                                                LEFT JOIN
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                                                        ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_58854988_condition_df = pandas.read_gbq(
    dataset_58854988_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_58854988_condition_df.head(5)
dataset_58854988_condition_df.to_csv("dataset_conditiondem_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "dementia and controls" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_58854988_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                SELECT
                    DISTINCT concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (
                        select
                            cast(cr.id as string) as id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                        WHERE
                            concept_id IN (
                                1586134,1585855,1740639
                            ) 
                            AND domain_id = 'SURVEY'
                    ) a 
                        ON (
                            c.path like CONCAT('%',
                        a.id,
                        '.%')) 
                    WHERE
                        domain_id = 'SURVEY' 
                        AND type = 'PPI' 
                        AND subtype = 'QUESTION'
                    )
            )  
            AND (
                answer.PERSON_ID IN (
                    SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (4182210) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 1 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 1 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_whole_genome_variant = 1 
                                )  
                            UNION
                            DISTINCT SELECT
                                distinct person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        age_at_consent BETWEEN 65 AND 120 
                                ) 
                                AND cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_whole_genome_variant = 1 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (372887) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (376337) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 1 
                                                        )
                                                    ) criteria 
                                                ) 
                                                AND cb_search_person.person_id NOT IN (
                                                    SELECT
                                                        criteria.person_id 
                                                    FROM
                                                        (SELECT
                                                            DISTINCT person_id,
                                                            entry_date,
                                                            concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                        WHERE
                                                            (
                                                                concept_id IN (
                                                                    SELECT
                                                                        DISTINCT c.concept_id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                    JOIN
                                                                        (
                                                                            select
                                                                                cast(cr.id as string) as id 
                                                                            FROM
                                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                            WHERE
                                                                                concept_id IN (4011630) 
                                                                                AND full_text LIKE '%_rank1]%'
                                                                        ) a 
                                                                            ON (
                                                                                c.path LIKE CONCAT('%.',
                                                                            a.id,
                                                                            '.%') 
                                                                            OR c.path LIKE CONCAT('%.',
                                                                            a.id) 
                                                                            OR c.path LIKE CONCAT(a.id,
                                                                            '.%') 
                                                                            OR c.path = a.id) 
                                                                        WHERE
                                                                            is_standard = 1 
                                                                            AND is_selectable = 1
                                                                        ) 
                                                                        AND is_standard = 1 
                                                                )
                                                            ) criteria 
                                                        ) 
                                                        AND cb_search_person.person_id NOT IN (
                                                            SELECT
                                                                criteria.person_id 
                                                            FROM
                                                                (SELECT
                                                                    DISTINCT person_id,
                                                                    entry_date,
                                                                    concept_id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                                WHERE
                                                                    (
                                                                        concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1740947) 
                                                                        OR  concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1740684) 
                                                                        OR  concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1384582) 
                                                                        OR  concept_id IN (1740558) 
                                                                        AND is_standard = 0  
                                                                        AND  value_source_concept_id IN (1740980)
                                                                    )) criteria ) 
                                                        )
                                                    )"""

dataset_58854988_survey_df = pandas.read_gbq(
    dataset_58854988_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_58854988_survey_df.head(5)
dataset_58854988_survey_df.to_csv("dataset_surveydem_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "dementia and controls" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_58854988_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (
                                    SELECT
                                        DISTINCT c.concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                    JOIN
                                        (
                                            select
                                                cast(cr.id as string) as id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                            WHERE
                                                concept_id IN (4182210) 
                                                AND full_text LIKE '%_rank1]%'
                                        ) a 
                                            ON (
                                                c.path LIKE CONCAT('%.',
                                            a.id,
                                            '.%') 
                                            OR c.path LIKE CONCAT('%.',
                                            a.id) 
                                            OR c.path LIKE CONCAT(a.id,
                                            '.%') 
                                            OR c.path = a.id) 
                                        WHERE
                                            is_standard = 1 
                                            AND is_selectable = 1
                                        ) 
                                        AND is_standard = 1 
                                )
                            ) criteria 
                        ) 
                        AND cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_whole_genome_variant = 1 
                        )  
                    UNION
                    DISTINCT SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                age_at_consent BETWEEN 65 AND 120 
                        ) 
                        AND cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_whole_genome_variant = 1 
                        ) 
                        AND cb_search_person.person_id NOT IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (372887) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 1 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 1 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (376337) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (4011630) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 1 
                                                        )
                                                    ) criteria 
                                                ) 
                                                AND cb_search_person.person_id NOT IN (
                                                    SELECT
                                                        criteria.person_id 
                                                    FROM
                                                        (SELECT
                                                            DISTINCT person_id,
                                                            entry_date,
                                                            concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                        WHERE
                                                            (
                                                                concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1740947) 
                                                                OR  concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1740684) 
                                                                OR  concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1384582) 
                                                                OR  concept_id IN (1740558) 
                                                                AND is_standard = 0  
                                                                AND  value_source_concept_id IN (1740980)
                                                            )) criteria ) 
                                                )"""

dataset_58854988_person_df = pandas.read_gbq(
    dataset_58854988_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_58854988_person_df.head(5)
dataset_58854988_person_df.to_csv("dataset_persondem_df.csv", index=False)

In [ ]:
person_condition_df = pd.merge(left=dataset_58854988_person_df, right=dataset_58854988_condition_df)
person_condition_df.shape

person_condition_df.to_csv("person_condition_dem_df.csv", index=False)

!grep dementia person_condition_dem_df.csv | cut -f1 -d "," | sort -u |  uniq > person_condition_dem_df_uniqid


## Control Cohort

In [ ]:
import pandas
import os

# This query represents dataset "controls_only" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_31844651_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN  (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            select
                                cast(cr.id as string) as id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                            WHERE
                                concept_id IN (
                                    378419, 4182210
                                ) 
                                AND full_text LIKE '%_rank1]%'
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    c_occurrence.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    age_at_consent BETWEEN 65 AND 120 
                            ) 
                            AND cb_search_person.person_id IN (
                                SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    has_whole_genome_variant = 1 
                            ) 
                            AND cb_search_person.person_id NOT IN (
                                SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (
                                                SELECT
                                                    DISTINCT c.concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                JOIN
                                                    (
                                                        select
                                                            cast(cr.id as string) as id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                        WHERE
                                                            concept_id IN (372887) 
                                                            AND full_text LIKE '%_rank1]%'
                                                    ) a 
                                                        ON (
                                                            c.path LIKE CONCAT('%.',
                                                        a.id,
                                                        '.%') 
                                                        OR c.path LIKE CONCAT('%.',
                                                        a.id) 
                                                        OR c.path LIKE CONCAT(a.id,
                                                        '.%') 
                                                        OR c.path = a.id) 
                                                    WHERE
                                                        is_standard = 1 
                                                        AND is_selectable = 1
                                                    ) 
                                                    AND is_standard = 1 
                                            )
                                        ) criteria 
                                    ) 
                                    AND cb_search_person.person_id NOT IN (
                                        SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    concept_id IN (
                                                        SELECT
                                                            DISTINCT c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    concept_id IN (43529272) 
                                                                    AND full_text LIKE '%_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                is_standard = 0 
                                                                AND is_selectable = 1
                                                            ) 
                                                            AND is_standard = 0 
                                                    )
                                                ) criteria 
                                            ) 
                                            AND cb_search_person.person_id NOT IN (
                                                SELECT
                                                    criteria.person_id 
                                                FROM
                                                    (SELECT
                                                        DISTINCT person_id,
                                                        entry_date,
                                                        concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                    WHERE
                                                        (
                                                            concept_id IN (
                                                                SELECT
                                                                    DISTINCT c.concept_id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                JOIN
                                                                    (
                                                                        select
                                                                            cast(cr.id as string) as id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                        WHERE
                                                                            concept_id IN (376337) 
                                                                            AND full_text LIKE '%_rank1]%'
                                                                    ) a 
                                                                        ON (
                                                                            c.path LIKE CONCAT('%.',
                                                                        a.id,
                                                                        '.%') 
                                                                        OR c.path LIKE CONCAT('%.',
                                                                        a.id) 
                                                                        OR c.path LIKE CONCAT(a.id,
                                                                        '.%') 
                                                                        OR c.path = a.id) 
                                                                    WHERE
                                                                        is_standard = 1 
                                                                        AND is_selectable = 1
                                                                    ) 
                                                                    AND is_standard = 1 
                                                            )
                                                        ) criteria 
                                                    ) 
                                                    AND cb_search_person.person_id NOT IN (
                                                        SELECT
                                                            criteria.person_id 
                                                        FROM
                                                            (SELECT
                                                                DISTINCT person_id,
                                                                entry_date,
                                                                concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                            WHERE
                                                                (
                                                                    concept_id IN (
                                                                        SELECT
                                                                            DISTINCT c.concept_id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                        JOIN
                                                                            (
                                                                                select
                                                                                    cast(cr.id as string) as id 
                                                                                FROM
                                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                                WHERE
                                                                                    concept_id IN (4011630) 
                                                                                    AND full_text LIKE '%_rank1]%'
                                                                            ) a 
                                                                                ON (
                                                                                    c.path LIKE CONCAT('%.',
                                                                                a.id,
                                                                                '.%') 
                                                                                OR c.path LIKE CONCAT('%.',
                                                                                a.id) 
                                                                                OR c.path LIKE CONCAT(a.id,
                                                                                '.%') 
                                                                                OR c.path = a.id) 
                                                                            WHERE
                                                                                is_standard = 1 
                                                                                AND is_selectable = 1
                                                                            ) 
                                                                            AND is_standard = 1 
                                                                    )
                                                                ) criteria 
                                                            ) ))
                                                ) c_occurrence 
                                            LEFT JOIN
                                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                                                    ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                                            LEFT JOIN
                                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                                                    ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                                            LEFT JOIN
                                                `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                                    ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                                            LEFT JOIN
                                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                                                    ON v.visit_concept_id = visit.concept_id 
                                            LEFT JOIN
                                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                                                    ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
                                            LEFT JOIN
                                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                                                    ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_31844651_condition_df = pandas.read_gbq(
    dataset_31844651_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_31844651_condition_df.head(5)
dataset_31844651_condition_df.to_csv("controlcondition_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "controls_only" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_31844651_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                SELECT
                    DISTINCT concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (
                        select
                            cast(cr.id as string) as id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                        WHERE
                            concept_id IN (
                                1586134,1585710,1740639,1585855
                            ) 
                            AND domain_id = 'SURVEY'
                    ) a 
                        ON (
                            c.path like CONCAT('%',
                        a.id,
                        '.%')) 
                    WHERE
                        domain_id = 'SURVEY' 
                        AND type = 'PPI' 
                        AND subtype = 'QUESTION'
                    )
            )  
            AND (
                answer.PERSON_ID IN (
                    SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                age_at_consent BETWEEN 65 AND 120 
                        ) 
                        AND cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_whole_genome_variant = 1 
                        ) 
                        AND cb_search_person.person_id NOT IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (372887) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 1 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 1 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (43529272) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 0 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 0 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (376337) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 1 
                                                        )
                                                    ) criteria 
                                                ) 
                                                AND cb_search_person.person_id NOT IN (
                                                    SELECT
                                                        criteria.person_id 
                                                    FROM
                                                        (SELECT
                                                            DISTINCT person_id,
                                                            entry_date,
                                                            concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                        WHERE
                                                            (
                                                                concept_id IN (
                                                                    SELECT
                                                                        DISTINCT c.concept_id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                    JOIN
                                                                        (
                                                                            select
                                                                                cast(cr.id as string) as id 
                                                                            FROM
                                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                            WHERE
                                                                                concept_id IN (4011630) 
                                                                                AND full_text LIKE '%_rank1]%'
                                                                        ) a 
                                                                            ON (
                                                                                c.path LIKE CONCAT('%.',
                                                                            a.id,
                                                                            '.%') 
                                                                            OR c.path LIKE CONCAT('%.',
                                                                            a.id) 
                                                                            OR c.path LIKE CONCAT(a.id,
                                                                            '.%') 
                                                                            OR c.path = a.id) 
                                                                        WHERE
                                                                            is_standard = 1 
                                                                            AND is_selectable = 1
                                                                        ) 
                                                                        AND is_standard = 1 
                                                                )
                                                            ) criteria 
                                                        ) ))"""

dataset_31844651_survey_df = pandas.read_gbq(
    dataset_31844651_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_31844651_survey_df.head(5)
dataset_31844651_survey_df.to_csv("controlsurvey_df.csv", index=False)

In [ ]:
import pandas
import os

# This query represents dataset "controls_only" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_31844651_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        age_at_consent BETWEEN 65 AND 120 
                ) 
                AND cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 
                ) 
                AND cb_search_person.person_id NOT IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (
                                    SELECT
                                        DISTINCT c.concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                    JOIN
                                        (
                                            select
                                                cast(cr.id as string) as id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                            WHERE
                                                concept_id IN (372887) 
                                                AND full_text LIKE '%_rank1]%'
                                        ) a 
                                            ON (
                                                c.path LIKE CONCAT('%.',
                                            a.id,
                                            '.%') 
                                            OR c.path LIKE CONCAT('%.',
                                            a.id) 
                                            OR c.path LIKE CONCAT(a.id,
                                            '.%') 
                                            OR c.path = a.id) 
                                        WHERE
                                            is_standard = 1 
                                            AND is_selectable = 1
                                        ) 
                                        AND is_standard = 1 
                                )
                            ) criteria 
                        ) 
                        AND cb_search_person.person_id NOT IN (
                            SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        concept_id IN (43529272) 
                                                        AND full_text LIKE '%_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    is_standard = 0 
                                                    AND is_selectable = 1
                                                ) 
                                                AND is_standard = 0 
                                        )
                                    ) criteria 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                concept_id IN (376337) 
                                                                AND full_text LIKE '%_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            is_standard = 1 
                                                            AND is_selectable = 1
                                                        ) 
                                                        AND is_standard = 1 
                                                )
                                            ) criteria 
                                        ) 
                                        AND cb_search_person.person_id NOT IN (
                                            SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (4011630) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) 
                                                                AND is_standard = 1 
                                                        )
                                                    ) criteria 
                                                ) )"""

dataset_31844651_person_df = pandas.read_gbq(
    dataset_31844651_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_31844651_person_df.head(5)
dataset_31844651_person_df.to_csv("controlperson_df.csv", index=False)

In [ ]:
!cut -f1 -d "," controlperson_df.csv | sort -u | uniq > control_ids

# Characterization of genes of interest in the Cohorts

In [ ]:
from datetime import datetime
import os
import pandas
start = datetime.now()

In [ ]:
bucket = os.getenv('WORKSPACE_BUCKET')
bucket

In [ ]:
genomic_location = os.getenv("CDR_STORAGE_PATH")
genomic_location

In [ ]:
#import pyspark
import hail as hl
hl.init(default_reference = "GRCh38")

In [ ]:
mt_wgs_path = os.getenv("WGS_EXOME_SPLIT_HAIL_PATH")
mt_wgs_path

In [ ]:
mt_wgs_path = "${wgs_path}/v7/wgs/short_read/snpindel/exome/splitMT/hail.mt"

In [ ]:
mt = hl.read_matrix_table(mt_wgs_path)
mt.count()

In [ ]:
test_intervals = ['chr:Start-End']

In [ ]:
mt = hl.filter_intervals(
    mt,
    [hl.parse_locus_interval(x,)
     for x in test_intervals])
mt.count()

In [ ]:
mt.locus.summarize()

In [ ]:
flagged_samples = "${wgs_path}/v7/wgs/short_read/snpindel/aux/relatedness/relatedness_flagged_samples.tsv"

In [ ]:
!gsutil -u $$GOOGLE_PROJECT cat $flagged_samples | head -n 3

In [ ]:
sample_to_remove = hl.import_table(flagged_samples, key="sample_id")

In [ ]:
mt = mt.anti_join_cols(sample_to_remove)
mt.count()

In [ ]:
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.90, keep = True)
mt.count()

In [ ]:
out_vcf = f'{bucket}/data/${Gene}.vcf.bgz'
out_vcf

In [ ]:
hl.export_vcf(mt, out_vcf, tabix=False)

In [ ]:
!bcftools view -S person_condition_${COHORT}_df_uniqid ${Gene}.vcf.bgz --force-samples > ${Gene}_${COHORT}.vcf
!grep CHROM ${Gene}_${COHORT}.vcf > ${Gene}_in${COHORT}
!for i in `cat ${Gene}_variants` ; do grep $i ${Gene}_${COHORT}.vcf >> ${Gene}_in${COHORT} ; done
!grep chr ${Gene}_in${COHORT} |  grep -v AC=0 | cut -f1-9

# Query All of Us to prepare data for Genotools, applying Genotools, and calculating allele frequency

## Preparing Genotype data for Genotools

In [ ]:
from datetime import datetime
import os
start = datetime.now()
bucket = os.getenv('WORKSPACE_BUCKET')
bucket

In [ ]:
genomic_location = os.getenv("CDR_STORAGE_PATH")
genomic_location

In [ ]:
array_plink_path_msa = f'{genomic_location}/microarray/plink'
array_plink_path_msa

In [ ]:
!gsutil -u $GOOGLE_PROJECT -m cp ${wgs_path}/v7/microarray/plink/* .

In [ ]:
## Create Sample IDs file and Keep sample IDs from the all array data
nano Allsampleids_PCA   
!awk '{print "0", $1}' Allsampleids_PCA > Allsampleids_PCA_FIDIID 
!cut -f2 Allsampleids_PCA_FIDIID -d " " | sed '/^$/d' > Allsampleids_PCA_FIDIID_spaceremoved
!awk '{print "0", $1}' Allsampleids_PCA_FIDIID_spaceremoved > Allsampleids_PCA_FIDIID_spaceremoved_fam_samples
!plink --bfile arrays --keep Allsampleids_PCA_FIDIID_spaceremoved_fam_samples --make-bed --out arrays_selected_samples_keep

## Apply Genotools 

In [ ]:
!pip install the-real-genotools

!git clone https://github.com/dvitale199/GenoTools
!cd GenoTools && pip install .

!/home/jupyter/.local/bin/genotools-download
!/home/jupyter/.local/bin/genotools --bfile /path/ --out /path/ --ancestry --ref_panel /path/ --ref_labels /path/

In [ ]:
import pandas as pd
import subprocess
import sys
import numpy as np
import os
import json
import plotly.express as px
import joblib
import numba
import sklearn
import pickle as pkl
from sklearn.preprocessing import LabelEncoder

In [ ]:
pkl_path = '${WORK_DIR}/GenoTools/genotools/nba_v1.pkl'

In [ ]:
pkl_in = open(pkl_path, 'rb')
pipe_clf = pkl.load(pkl_in)
pkl_in.close()

In [ ]:
ref_pca_path = '${WORK_DIR}/arrays_selected_samples_keep_genotools_labeled_ref_pca.txt'

ref_pca = pd.read_csv(ref_pca_path, sep='\t')
print(ref_pca.head())
print(ref_pca.shape)

y = ref_pca['label']
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
proj_pca_path = '${WORK_DIR}/arrays_selected_samples_keep_genotools_projected_new_pca.txt'

proj_pca = pd.read_csv(proj_pca_path, sep='\t')
print(proj_pca.head())
print(proj_pca.shape)

In [ ]:
proj_for_pred = proj_pca.drop(columns=['FID','IID','label'], axis=1)
print(proj_for_pred.head())
print(proj_for_pred.shape)

In [ ]:
y_pred = pipe_clf.predict(proj_for_pred)

In [ ]:
y_pred = pd.Series(le.inverse_transform(y_pred))
print(y_pred.value_counts())
print(y_pred.head())

In [ ]:
proj_ids = proj_pca[['FID','IID']]
print(proj_ids.head())

In [ ]:
labels = pd.concat([proj_ids,y_pred], axis=1)
labels.columns = ['FID','IID','label']
print(labels.head())
print(labels.shape)

In [ ]:
labels.to_csv('${WORK_DIR}/arrays_selected_samples_keep_genotools_predicted_labels.txt', sep='\t', index=False)
!cat arrays_selected_samples_keep_genotools_predicted_labels.txt

In [ ]:
## Seperating AD, Dementia and Controls cohorts
!awk 'NR==FNR{a[$1]; next} $2 in a' ${COHORT}_sampleid arrays_selected_samples_keep_genotools_predicted_labels.txt > filtered_${COHORT}_sampleid_genotools.txt

In [ ]:
## Count the number of individuals in each ancestry 
!awk '$3 == "${ANCESTRY}" {count++} END {print count}' filtered_${COHORT}_sampleid_genotools.txt

## Using Genotools for calculating allele frequencies

In [ ]:
!awk '{print $2 "\t" $2 "\t" $3}' filtered_${COHORT}_sampleid_genotools.txt > filtered_${COHORT}_sampleid_genotoolsFIDIID.txt
!grep ${ANCESTRY} filtered_${COHORT}_sampleid_genotoolsFIDIID.txt> filtered_${COHORT}_sampleid_genotools_${ANCESTRY}FIDIID
!plink2 --vcf ${Gene}.vcf.bgz --make-bed --out ${Gene}_largeplink  --set-missing-var-ids @:#\$r-\$a --double-id --new-id-max-allele-len 80
!plink --merge-list merge_list_genes --make-bed --out ALL_GENES_plink
!plink2 --bfile ALL_GENES_plink  --make-bed --out ${COHORT}_${ANCESTRY}_ALLgenes --new-id-max-allele-len 80 --double-id --keep filtered_${COHORT}_sampleid_genotools_${ANCESTRY}FIDIID --set-missing-var-ids @:#\$r-\$a
!plink --bfile ${COHORT}_${ANCESTRY}_ALLgenes --extract All_variants_for_freq_genotools --freq --out FREQ_${COHORT}_${ANCESTRY}_genotools
!cat FREQ_${COHORT}_${ANCESTRY}_genotools.frq

# Query All of Us for APOE genotyping and defining ancestry for each genotype

## AOPE Genotyping

In [ ]:
from datetime import datetime
import os 
import pandas as pd

In [ ]:
start = datetime.now()
bucket = os.getenv("WORKSPACE_BUCKET")
bucket

In [ ]:
import hail as hl
hl.init(default_reference='GRCh38', idempotent=True)

In [ ]:
mt_wgs_path = os.getenv("WGS_EXOME_SPLIT_HAIL_PATH")
mt_wgs_path

In [ ]:
mt_wgs_path = "${wgs_path}/v7/wgs/short_read/snpindel/exome/splitMT/hail.mt"

In [ ]:
mt = hl.read_matrix_table(mt_wgs_path)

In [ ]:
intervals = ['chr19:44908684-44908685', 'chr19:44908821-44908823']
filt_mt = hl.filter_intervals(mt, [hl.parse_locus_interval(x, reference_genome='GRCh38') for x in intervals])
filt_mt.show()

In [ ]:
bi = filt_mt.filter_rows(hl.len(filt_mt.alleles) == 2)
bi = bi.annotate_rows(a_index=1, was_split=False)
multi = filt_mt.filter_rows(hl.len(filt_mt.alleles) > 2)

In [ ]:
split = hl.split_multi_hts(multi,
                               keep_star=False,
                               left_aligned=False,
                               vep_root='vep',
                               permit_shuffle=False)

In [ ]:
filt_mt = split.union_rows(bi)
filt_mt.show()

In [ ]:
out_path = f'{bucket}/data/test_plink'

In [ ]:
hl.export_plink(filt_mt, out_path, ind_id = filt_mt.s)

In [ ]:
!mkdir plink_files

In [ ]:
!gsutil cp '{bucket}/data/test_plink*' plink_files/

In [ ]:
!plink --bfile plink_files/test_plink --snps chr19:44908684:T:C,chr19:44908822:C:T --make-bed --out plink_files/apoe_snps

In [ ]:
!plink --bfile plink_files/apoe_snps --recode compound-genotypes --out plink_files/apoe_snps_recode

In [ ]:
%%writefile APOE_genotypes_PLINK_ped.py
#!/bin/env python

# Determine APOE genotypes from PLINK output
    # January 2021
    # Mary B. Makarious, Makayla Portley, and Cornelis Blauwendraat (LNG/NIA/NINDS/NIH)
    # Script usage:
        # python APOE_genotypes_PLINK_ped.py -i INPUT.ped -o OUTPUT_NAME

## APOE Information
# https://www.snpedia.com/index.php/APOE

    # |          APOE GENO         	| rs429358 	| rs7412 	|             COMBINED             	|
    # |:--------------------------:	|:--------:	|:------:	|:--------------------------------:	|
    # |            e1/e1           	|    CC    	|   TT   	|               CC_TT              	|
    # |            e1/e2           	|    CT    	|   TT   	|          CT_TT or TC_TT          	|
    # |            e1/e4           	|    CC    	|   CT   	|          CC_CT or CC_TC          	|
    # |            e2/e2           	|    TT    	|   TT   	|               TT_TT              	|
    # |            e2/e3           	|    TT    	|   TC   	|          TT_TC or TT_CT          	|
    # | e2/e4 or e1/e3 (Ambiguous) 	|    TC    	|   TC   	| TC_TC or CT_CT or TC_CT or CT_TC 	|
    # |            e3/e3           	|    TT    	|   CC   	|               TT_CC              	|
    # |            e3/e4           	|    TC    	|   CC   	|          TC_CC or CT_CC          	|
    # |            e4/e4           	|    CC    	|   CC   	|               CC_CC              	|

# Import the necessary packages
import numpy as np
import pandas as pd
import sys
from functools import reduce
import argparse

# Initialize parser and add arguments
parser = argparse.ArgumentParser()
parser.add_argument("--input", "-i", help="Input file name (with suffix)")
parser.add_argument("--output", "-o", help="Desired output name (without suffix)")
args = parser.parse_args()

# Read in the .ped file and force column names
header_text = ["FID", "IID", "PAT", "MAT", "SEX", "PHENO", "rs429358", "rs7412"]
input_ped_df = pd.read_csv(args.input, sep = " ", header=None, names=header_text)

# Make a combined column, gluing the genotypes from the rs429358 and rs7412 columns
input_ped_df['rs429358_rs7412'] = input_ped_df['rs429358'].astype(str)+'_'+input_ped_df['rs7412']

# Initialize a dictionary with the genotypes to search what genotype the alleles generate
apoe_genotypes_dict = {
    'CC_TT' : 'e1/e1',
    'CT_TT' : 'e1/e2',
    'TC_TT' : 'e1/e2',
    'CC_CT' : 'e1/e4',
    'CC_TC' : 'e1/e4',
    'TT_TT' : 'e2/e2',
    'TT_TC' : 'e2/e3',
    'TT_CT' : 'e2/e3',
    'TC_TC' : 'e2/e4 or e1/e3',
    'CT_CT' : 'e2/e4 or e1/e3',
    'TC_CT' : 'e2/e4 or e1/e3',
    'CT_TC' : 'e2/e4 or e1/e3',
    'TT_CC' : 'e3/e3',
    'TC_CC' : 'e3/e4',
    'CT_CC' : 'e3/e4',
    'CC_CC' : 'e4/e4'
}

# Map the combined column to the dictionary to extract the genotypes
input_ped_df['APOE_GENOTYPE'] = input_ped_df['rs429358_rs7412'].map(apoe_genotypes_dict)

# If any of the combined alleles weren't in the dictionary, the dataframe now has NaN values
# This happens if you have a 0 or missingness somewhere, resulting in an unsure genotype call
# Replace these with something more useful, and state the APOE genotype as "unknown"
input_ped_df.replace(np.nan, 'unknown', regex=True, inplace=True)

# Make a file of just the FID, IID, SEX, PHENO, and APOE genotype
subset_geno_df = input_ped_df.drop(columns=['PAT', 'MAT', 'rs429358', 'rs7412'])

## Generate counts
# Generate APOE genotype counts and percentages for entire dataset
counts_df = pd.DataFrame(subset_geno_df['APOE_GENOTYPE'].value_counts().reset_index())
counts_df.columns = ['APOE_GENOTYPE', 'TOTAL_COUNT']
counts_df['TOTAL_PERCENT'] = counts_df['TOTAL_COUNT'] / subset_geno_df.shape[0] * 100

# Separate out into cases, controls, and missing phenotypes
    # This assumes controls=1 and cases=2 (missing is -9)

# Subset by phenotype
missing_pheno_df = subset_geno_df[subset_geno_df['PHENO'] == -9]
controls_df = subset_geno_df[subset_geno_df['PHENO'] == 1]
cases_df = subset_geno_df[subset_geno_df['PHENO'] == 2]

# Generate APOE genotype counts and percentages for missing phenotypes
missing_pheno_counts_df = pd.DataFrame(missing_pheno_df['APOE_GENOTYPE'].value_counts().reset_index())
missing_pheno_counts_df.columns = ['APOE_GENOTYPE', 'MISSING_PHENO_COUNT']
missing_pheno_counts_df['MISSING_PHENO_PERCENT'] = missing_pheno_counts_df['MISSING_PHENO_COUNT'] / missing_pheno_df.shape[0] * 100

# Generate APOE genotype counts and percentages for controls
controls_counts_df = pd.DataFrame(controls_df['APOE_GENOTYPE'].value_counts().reset_index())
controls_counts_df.columns = ['APOE_GENOTYPE', 'CONTROLS_COUNT']
controls_counts_df['CONTROLS_PERCENT'] = controls_counts_df['CONTROLS_COUNT'] / controls_df.shape[0] * 100

# Generate APOE genotype counts and percentages for cases
cases_counts_df = pd.DataFrame(cases_df['APOE_GENOTYPE'].value_counts().reset_index())
cases_counts_df.columns = ['APOE_GENOTYPE', 'CASES_COUNT']
cases_counts_df['CASES_PERCENT'] = cases_counts_df['CASES_COUNT'] / cases_df.shape[0] * 100

# Merge the dataframes together for final summary counts file
dataframes_tomerge = [counts_df, missing_pheno_counts_df, controls_counts_df, cases_counts_df]
merged_summary_df = reduce(lambda left,right: pd.merge(left,right,on='APOE_GENOTYPE'), dataframes_tomerge)

## Export
complete_df_output = args.output + ".APOE_GENOTYPES.csv"
counts_df_output = args.output + ".APOE_SUMMARY.csv"

# Save out the complete dataframe as a .csv
print(f"Your complete genotype file has been saved here: {complete_df_output}")
subset_geno_df.to_csv(complete_df_output, index=False)

# Save out the counts as a .csv
print(f"The summary counts have been saved here: {counts_df_output}")
merged_summary_df.to_csv(counts_df_output, index=False)

# Done!
print("Thanks!")

In [ ]:
%%bash
python APOE_genotypes_PLINK_ped.py -i plink_files/apoe_snps_recode.ped -o plink_files/apoe_snps_test

In [ ]:
apoe = pd.read_csv('plink_files/apoe_snps_test.APOE_GENOTYPES.csv')
apoe.head()

In [ ]:
### Create files including sample IDs of AD, Dementia, and Controls
nano AD_SAMPELID
nano Dementia_SAMPELID
nano Controls_SAMPLEID

In [ ]:
### Seperating genotypes for AD, Dementia and Controls cohorts
!for i in `cat AD_SAMPELID` ; do grep $i plink_files/apoe_snps_test.APOE_GENOTYPES.csv >> AD_APOE ; done
!for i in `cat Dementia_SAMPELID` ; do grep $i plink_files/apoe_snps_test.APOE_GENOTYPES.csv >> Dementia_APOE ; done
!for i in `cat Controls_SAMPLEID` ; do grep $i plink_files/apoe_snps_test.APOE_GENOTYPES.csv >> Controls_APOE ; done

In [ ]:
### Count the number of individuals in each genotype
with open('${COHORT}', 'r') as file:
    # Read the content
    content = file.read()
    # Count occurrences of 'genotype'
    count = content.count('${GENOTYPE}')

# Print the count
print("Word count of 'genotype' in COHORT file:", count)

## Using Genotools to define ancestry for each genotype

In [ ]:
!cut -f2- filtered_${COHORT}sampleid_genotools.txt > forAPOE_filtered_${COHORT}sampleid_genotools.txt
!cut -d ',' -f 2,5,6 ${COHORT}_APOE > ${COHORT}_APOE_THREECOLUMN

In [ ]:
import pandas as pd

# Read the first file
df1 = pd.read_csv('forAPOE_filtered_${COHORT}sampleid_genotools.txt', header=None, delimiter='\t', names=['Sample_ID', 'Ancestry'])

# Read the second file
df2 = pd.read_csv('${COHORT}_APOE_THREECOLUMN', header=None, delimiter=',', names=['Sample_ID', 'Data1', 'Data2'])

# Merge the two dataframes on the sample ID column
merged_df = pd.merge(df1, df2, on='Sample_ID', how='inner')

# Save the merged dataframe to a new file
merged_df.to_csv('${COHORT}_APOE_ancestry', index=False, sep='\t')

In [ ]:
import pandas as pd

# Read the file into a DataFrame
df = pd.read_csv('${COHORT}_APOE_ancestry', sep='\t')

# Count the occurrences where 'Ancestry' is 'ANCESTRY' and 'Data2' is 'GENOTYPE'
count = len(df[(df['Ancestry'] == '${ANCESTRY}') & (df['Data2'] == '${GENOTYPE}')])

print("Number of rows with 'ANCESTRY' and 'GENOTYPE' data:", count)

# Query All of Us for demographic and phenotypic data

In [ ]:
## Number of Female/Male
!awk -F',' '$3 ~ /Female/' unique_${COHORT}sampleid_person | wc -l
!awk -F',' '$3 ~ /Male/' unique_${COHORT}sampleid_person | wc -l
# Checking missing data
!awk -F',' '$3 !~ /(Female|Male)/ {print $3}' unique_${COHORT}sampleid_person

In [ ]:
## Age at onset for Female/Male
!awk -F',' '$3 ~ /Male/' unique_${COHORT}sampleid_person > ${COHORT}male
!awk -F',' '{print $4}' ${COHORT}male
!awk -F',' '$3 ~ /Female/' unique_${COHORT}sampleid_person > ${COHORT}female
!awk -F',' '{print $4}' ${COHORT}female
## Checking missing data
!awk -F',' '$3 !~ /(Female|Male)/ {print $7}' unique_${COHORT}sampleid_person

In [ ]:
## Number of Homozygous/Heterozygous
!bcftools query -i 'POS == ${}' -f "[%GT\n]" ${Gene}_${COHORT}.vcf | awk '/0\/0/ {hom_ref++} /0\/1/ {het++} /1\/1/ {hom_alt++} END {print "Homozygous Reference:", hom_ref, "\nHeterozygous:", het, "\nHomozygous Alternate:", hom_alt}'

In [ ]:
## Check data for one sample ID
!grep CHROM ${Gene}_${COHORT}.vcf > ${Gene}_${COHORT}
!grep Position ${Gene}_${COHORT}.vcf >> ${Gene}_${COHORT}
!awk -F '\t' 'NR==1 {for (i=10; i<=NF; i++) sampleIds[i-9]=$i; print "Sample_IDs"; next} {for (i=10; i<=NF; i++) {split($i, gt, ":"); if (gt[1] == "0/1") print sampleIds[i-9]}}'  ${Gene}_${COHORT}
!grep ${Sample_IDs} dataset_person_df.csv| head -30
!gsutil -u $GOOGLE_PROJECT cat "${wgs_path}/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv" | grep ${Sample_IDs}
## Check data for Sample IDs
!grep ${Sample_IDs} dataset_condition_df.csv
!grep ${Sample_IDs} dataset_survey_df.csv
!grep ${Sample_IDs} dataset_person_df.csv

In [ ]:
## Check phenotying data
import pandas as pd

# Load the dataset
file_path = 'dataset_condition_pheno_df.csv'
df = pd.read_csv(file_path)

# List of person_ids to search for
person_ids = [Sample IDs]

# Filter the dataset based on the person_ids
filtered_df = df[df['person_id'].isin(person_ids)]

# Select only the source_concept_name column
result_df = filtered_df[[df.columns[14]]]

# To display all rows in the output
pd.set_option('display.max_rows', None)
print(result_df)

# Save the result to a CSV file
result_df.to_csv('SampleIDs_source_concept_names.csv', index=False)

In [ ]:
## Check phenotying data
import pandas as pd

# Load the dataset
file_path = 'dataset_survey_pheno_df.csv'
df = pd.read_csv(file_path)

# List of person_ids to search for
person_ids = [Sample IDs]

# Filter the dataset based on the person_ids
filtered_df = df[df['person_id'].isin(person_ids)]

# Select only the standard_concept_name column
result_df = filtered_df[['answer']]

# To display all rows in the output
pd.set_option('display.max_rows', None)
print(result_df)

# Save the result to a CSV file
result_df.to_csv('Sample IDs_answer_names.csv', index=False)

# Query All of Us for resilience and protective variants

In [ ]:
from datetime import datetime
import os
start = datetime.now()

In [ ]:
bucket = os.getenv('WORKSPACE_BUCKET')
bucket

In [ ]:
genomic_location = os.getenv("CDR_STORAGE_PATH")
genomic_location

In [ ]:
import hail as hl

In [ ]:
hl.init(default_reference = "GRCh38")

In [ ]:
vds_srwgs_path = os.getenv("WGS_VDS_PATH")
vds_srwgs_path

In [ ]:
vds_srwgs_path = "${wgs_path}/v7/wgs/short_read/snpindel/vds/hail.vds"

In [ ]:
vds = hl.vds.read_vds(vds_srwgs_path)

In [ ]:
vds.reference_data.count()

In [ ]:
vds.reference_data.describe()

In [ ]:
vds.variant_data.count()

In [ ]:
vds.variant_data.describe()

In [ ]:
test_intervals = ['chr${}:${START}-${END}',chr${}:${START}-${END}']

In [ ]:
vds = hl.vds.filter_intervals(
    vds,
    [hl.parse_locus_interval(x,)
     for x in test_intervals])

In [ ]:
vds.variant_data.count()

In [ ]:
mt = vds.variant_data.annotate_entries(AD = hl.vds.local_to_global(vds.variant_data.LAD, 
                                                                   vds.variant_data.LA, 
                                                                   n_alleles=hl.len(vds.variant_data.alleles), 
                                                                   fill_value=0, number='R'))

In [ ]:
mt = mt.annotate_entries(GT = hl.vds.lgt_to_gt(mt.LGT, mt.LA))

In [ ]:
mt = mt.transmute_entries(FT = hl.if_else(mt.FT, "PASS", "FAIL"))

In [ ]:
mt = hl.vds.to_dense_mt(hl.vds.VariantDataset(vds.reference_data, mt))

In [ ]:
filt_mt = hl.filter_intervals(mt, [hl.parse_locus_interval(x, reference_genome='GRCh38') for x in test_intervals])
filt_mt.show()

In [ ]:
out_vcf_Protective_variants = f'{bucket}/data/Protective_variants.vcf.bgz'
out_vcf_Protective_variants

In [ ]:
hl.export_vcf(filt_mt, out_vcf_Protective_variants, tabix=False)

In [ ]:
!gsutil cp gs://fc-secure-477e063b-f55d-4493-8377-a24ef9591025/data/Protective_variants1.vcf.bgz .

In [ ]:
# Define genotypes
genotypes = {
    "e1/e1": [],
    "e1/e2": [],
    "e1/e4": [],
    "e2/e2": [],
    "e2/e3": [],
    "e3/e3": [],
    "e3/e4": [],
    "e4/e4": [],
    "e2/e4 or e1/e3": []  
}

# Read the file
file_path = "${COHORT}_APOE_ancestry"
with open(file_path, "r") as file:
    # Skip the header
    next(file)
    
    # Process each line
    for line in file:
        fields = line.strip().split("\t")
        sample_id = fields[0]
        genotype_field = fields[3]
        
        # Split the genotype field
        genotypes_list = genotype_field.split("_")
        
        # Add the sample ID to the appropriate genotype lists
        for genotype in genotypes_list:
            if genotype in genotypes:
                genotypes[genotype].append(sample_id)

# Print the total number of sample IDs for each genotype
for genotype, sample_ids in genotypes.items():
    print(f"Genotype: {genotype}, Total Sample IDs: {len(sample_ids)}")

# Save the sample IDs for the combined genotype to a separate file
combined_genotype = "e2/e4 or e1/e3"
sample_ids = set(genotypes[combined_genotype])

output_file_path = f"APOE_{combined_genotype.replace(' or ', '_').replace('/', '')}_only_sampleids.txt"
with open(output_file_path, "w") as output_file:
    for sample_id in sample_ids:
        output_file.write(f"{sample_id}\n")


In [ ]:
import subprocess

# Define the list of genotypes to use as sample ID files
genotypes_files = [
    "APOE_e1e1_only_sampleids.txt",
    "APOE_e1e2_only_sampleids.txt",
    "APOE_e1e4_only_sampleids.txt",
    "APOE_e2e2_only_sampleids.txt",
    "APOE_e2e3_only_sampleids.txt",
    "APOE_e3e3_only_sampleids.txt",
    "APOE_e3e4_only_sampleids.txt",
    "APOE_e4e4_only_sampleids.txt",
    "APOE_e2e4_e1e3_only_sampleids.txt"
]

# Input VCF file
vcf_file = "Protective_variants.vcf.bgz"

# Iterate over each genotype file
for genotype_file in genotypes_files:
    # Define the output file name
    output_file_name = f"{genotype_file}_Protective_variants.vcf"

    # Run bcftools to filter the VCF file
    cmd = f"bcftools view -S {genotype_file} {vcf_file} --force-samples > {output_file_name}"
    !{cmd}

    # Print the name of the file created
    print(f"Filtered VCF file created: {output_file_name}")

In [ ]:
import pysam

# List of VCF files to process
vcf_files = [
    "APOE_e1e1_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e1e2_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e1e4_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e2e2_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e2e3_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e3e3_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e3e4_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e4e4_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e2e4_e1e3_only_sampleids.txt_Protective_variants.vcf"
]

for vcf_file in vcf_files:
    with pysam.VariantFile(vcf_file) as vcf:
        sample_ids = list(vcf.header.samples)
    
        for record in vcf:
            variant_info = f"{record.chrom}:{record.pos}:{record.ref}:{record.alts[0]}"
            print(f"VCF File: {vcf_file}, Variant: {variant_info}")
        
            variant_sample_ids = {sample_id: False for sample_id in sample_ids}
        
            for sample_id in sample_ids:
                genotype_field = record.samples[sample_id]["GT"]
                allele1, allele2 = map(int, str(genotype_field).strip('()').split(', '))
            
                if allele1 == 0 and allele2 == 1:
                    variant_sample_ids[sample_id] = True
        
            for sample_id, has_genotype in variant_sample_ids.items():
                if has_genotype:
                    print(f"VCF File: {vcf_file}, Variant: {variant_info}, Sample ID: {sample_id}")

In [ ]:
import pysam

# Dictionary to store sample IDs and their corresponding ancestry
ancestry_dict = {}

# Read the '${COHORT}_APOE_ancestry' file and populate the ancestry dictionary
with open('${COHORT}_APOE_ancestry', 'r') as ancestry_file:
    for line in ancestry_file:
        parts = line.strip().split()
        sample_id = parts[0]
        ancestry = parts[1]
        ancestry_dict[sample_id] = ancestry

# List of VCF files to process
vcf_files = [
    "APOE_e1e1_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e1e2_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e1e4_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e2e2_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e2e3_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e3e3_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e3e4_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e4e4_only_sampleids.txt_Protective_variants.vcf",
    "APOE_e2e4_e1e3_only_sampleids.txt_Protective_variants.vcf"
]

# Open a new file to save the results
with open('results.csv', 'w') as results_file:
    # Write header to the results file
    results_file.write("Gene, Genotype, Sample ID, Ancestry, Chrom:Pos, Ref Allele, Alt Allele\n")
    
    # Iterate over each VCF file
    for vcf_file in vcf_files:
        with pysam.VariantFile(vcf_file) as vcf:
            sample_ids = list(vcf.header.samples)
        
            for record in vcf:
                variant_info = f"{record.chrom}:{record.pos}:{record.ref}:{record.alts[0]}"
                
                variant_sample_ids = {sample_id: False for sample_id in sample_ids}
                
                for sample_id in sample_ids:
                    genotype_field = record.samples[sample_id]["GT"]
                    if genotype_field is None or '.' in genotype_field:
                        # Skip samples with missing or unphased genotypes
                        continue
                    
                    try:
                        allele1, allele2 = map(int, str(genotype_field).strip('()').split(', '))
                    except ValueError:
                        # Skip samples with invalid genotype format
                        continue
                
                    if allele1 == 0 and allele2 == 1:
                        variant_sample_ids[sample_id] = True
            
                for sample_id, has_genotype in variant_sample_ids.items():
                    if has_genotype:
                        ancestry = ancestry_dict.get(sample_id, 'Ancestry not found')
                        gene = str(vcf_file).split("_")[0]
                        genotype = str(vcf_file).split("_")[1]
                        chrom_pos = f"{record.chrom}:{record.pos}"
                        ref, alt = record.ref, record.alts[0]
                        result_line = f"{gene}, {genotype}, {sample_id}, {ancestry}, {chrom_pos}, {ref}, {alt}\n"

                        results_file.write(result_line)


In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('results.csv')

# Strip whitespace from column names
df.columns = df.columns.str.strip()

# Group the data by variation (Chrom:Pos)
grouped = df.groupby('Chrom:Pos')

# Define the total counts
totals = {
    'Total': ${COUNT},
    'EUR': ${COUNT},
    'AFR': ${COUNT},
    'AMR': ${COUNT},
    'EAS': ${COUNT},
    'SAS': ${COUNT},
    'MDE': ${COUNT},
    'AJ': ${COUNT},
    'AAC': ${COUNT},
    'CAS': ${COUNT}
}

# Initialize an empty string to store the table
table_str = ""

# Iterate over each unique variant
for variant, data in grouped:
    # Count the number of occurrences of each genotype for each ancestry
    counts = data.pivot_table(index='Genotype', columns='Ancestry', values='Sample ID', aggfunc='count', fill_value=0)
    
    # Add a total row with the provided total numbers
    counts.loc['Total'] = [totals[col.strip()] if col != 'Total' else totals[col] for col in counts.columns]
    
    # Convert counts to strings
    counts_str = counts.astype(str)
    
    # Initialize a new DataFrame to hold the formatted values
    result_df = pd.DataFrame(index=counts_str.index, columns=counts_str.columns)
    
    # Fill the new DataFrame with the formatted values
    for col in counts_str.columns:
        result_df[col] = counts_str[col] + ('' if col == 'Total' else ' (' + ((counts[col] / counts.loc['Total', col]) * 100).round(2).astype(str) + '%)')
    
    # Remove the percentage from the "Total" row
    result_df.loc['Total'] = counts_str.loc['Total']
    
    # Add the variant header to the table
    table_str += f"{variant}\n"
    
    # Add the header row for ancestries
    table_str += "\t" + "\t".join(result_df.columns) + "\n"
    
    # Add the genotype counts for each ancestry
    for genotype, row in result_df.iterrows():
        table_str += f"{genotype}\t" + "\t".join(row) + "\n"
    
    # Add a blank line between variants
    table_str += "\n"

# Save the table to a text file
with open('table_summary.txt', 'w') as file:
    file.write(table_str)
